In [1]:
import pandas as pd
import logging

from split_data_type import SplitDataType
from pipeline import TreeSimilarPipeline
from feature_manager import FeatureManager

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
target_column = "SalePrice"

train_df = pd.read_csv("../HousePrice_train.csv")
train_X = train_df.drop(columns=[target_column])
train_y = train_df[target_column]
test_X = pd.read_csv("../HousePrice_test.csv")
test_y = pd.read_csv("../sample_submission.csv")[target_column]

In [4]:
FeatureManager.get_smallest_category(train_X)

,value,frequency(%)
column,,
Utilities,NoSeWa,0.068493
RoofMatl,Metal,0.068493
Exterior1st,AsphShn,0.068493
Condition2,PosA,0.068493
HeatingQC,Po,0.068493
Heating,Floor,0.068493
ExterCond,Po,0.068493
Exterior2nd,Other,0.068493
Functional,Sev,0.068493


In [5]:
FeatureManager.get_high_entropy(train_X)

['Id']

In [6]:
train_X = TreeSimilarPipeline(train_X, SplitDataType.TRAIN).build()

DEBUG:root:TRAIN - Function: _drop_not_needed
Size: (1460, 80)
Columns with null: LotFrontage, Alley, MasVnrType, MasVnrArea, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature

DEBUG:root:TRAIN - Function: _fill_null
Size: (1460, 80)
Columns with null: None

DEBUG:root:TRAIN - Function: _encode
Size: (1460, 80)
Columns with null: None

DEBUG:root:TRAIN - Function: _normalize
Size: (1460, 80)
Columns with null: None

DEBUG:root:TRAIN - Function: _drop_high_correlation
Size: (1460, 80)
Columns with null: None



In [7]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,-1,Reg,Lvl,AllPub,...,0,0,-1,-1,-1,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,-1,Reg,Lvl,AllPub,...,0,0,-1,-1,-1,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,-1,IR1,Lvl,AllPub,...,0,0,-1,-1,-1,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,-1,IR1,Lvl,AllPub,...,0,0,-1,-1,-1,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,-1,IR1,Lvl,AllPub,...,0,0,-1,-1,-1,0,12,2008,WD,Normal
